In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'heat stress guidlines interventions mitigation strategies' 
}


In [2]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 5

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))


  0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
results.to_csv('article_info.csv')

In [4]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [5]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [400]>


In [6]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [7]:
#scraping function

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome() #initialize web-finder
    main_page_url = 'https://sci-hub.ru/' #open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    try:
        # Find download link
        download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
        onclick_attribute = download_button.get_attribute('onclick')
    except:
        driver.quit()
        return url_to_search

    
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
        file.write(response.content)
    
    driver.quit()

    return None



In [8]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE

import pandas as pd
import os

data = pd.read_csv('article_info.csv')
title = data['title']
links = data['link']
author = ['author'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []


In [9]:
#function to call on all links

for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['link'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['link'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

PDF already downloaded
ERROR FOR https://www.jstage.jst.go.jp/article/indhealth/51/1/51_2012-0089/_article/-char/ja/
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
ERROR FOR https://journals.uni-lj.si/aas/article/view/12783
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
ERROR FOR https://jamanetwork.com/journals/jamainternalmedicine/article-abstract/413470
PDF already downloaded
ERROR FOR https://researchportal.port.ac.uk/files/3398222/Heat_acclimation.pdf
PDF already downloaded
PDF already downloaded
ERROR FOR https://books.google.com/books?hl=en&lr=&id=D9pNrH6U0oYC&oi=fnd&pg=PR7&dq=heat+stress+guidlines+interventions+mitigation+strategies&ots=Uwmb8JjYGG&sig=mP62VMTdXLhQoeU9Ji7EMZhBSFU
PDF already downloaded
PDF already downloaded
PDF alr

Scrape Discussion

In [10]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\Gur Levy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join

mypath = 'C:\\Users\\celia\\OneDrive - UvA\\semester 3\\DE\\trashmasters\\scholar scraper\\pdfs'

pdf_folder_content = [f for f in listdir(mypath) if isfile(join(mypath, f))]

all_pdf_texts = []

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')
	
    reader = PdfReader(f'{mypath}/{pdf}')

    n_pages = len(reader.pages)
	
    print(f'{pdf} has {n_pages} pages!')

    all_text = []

    for i in range(0,n_pages):  
      page_text = reader.pages[i].extract_text()
      all_text.append(page_text)

    result_string = ' '.join(all_text)
    all_pdf_texts.append(result_string)

##SWITCH TO A DICTIONARY INSTEAD OF A LIST
# 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\celia\\OneDrive - UvA\\semester 3\\DE\\trashmasters\\scholar scraper\\pdfs'

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re


from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel

lemmatizer = WordNetLemmatizer()

additional_stopwords = ['et', 'al','b', 'fw', 'e'] 

def preprocess_text(text):
    #YOUR CODE HERE

    # removes special characters and punctuation (STEP 1: Text cleaning)
    text = re.sub(r'[^A-Za-z\s]', '', text) 
    # remove additional white spaces (STEP 1: Text cleaning)
    text = re.sub(r'\s+', ' ', text).strip() 

    #put text in lowercases (STEP 2: Case normalization)
    text = text.lower()

    #STEP 3: tokenization
    tokens = word_tokenize(text)

    #STEP 4: removing stopwords:
    stopwords_list = set(stopwords.words('english')).union(additional_stopwords)
    tokens = [token for token in tokens if token not in stopwords_list]

    #STEP 5: lemmeatization: 
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_words

In [ ]:
processed_text = [preprocess_text(text) for text in all_pdf_texts]

corpus = corpora.Dictionary(processed_text)
doc_term_matrix = [corpus.doc2bow(text) for text in processed_text]

num_topics = 45

lda_model = LdaModel(doc_term_matrix, id2word = corpus, num_topics = num_topics)



In [ ]:
from collections import Counter

document_topics = [lda_model.get_document_topics(doc) for doc in doc_term_matrix]
all_topics = [topic[0] for topics in document_topics for topic in topics]

topics_nb = Counter(all_topics)

most_nb_topics = [topic for topic, count in topics_nb.most_common()]
print("The most common topics and the number of instances are:")
for topic, x in topics_nb.most_common():
    print(f"Topic {topic +1}: Count {x}")
for topic in most_nb_topics:
    topics_words = lda_model.show_topic(topic)
    most_words = [word for word, _ in topics_words]
    print(f"Most used words in Topic {topic +1}: {', '.join(most_words)}")

The most common topics and the number of instances are:
Topic 16: Count 6
Topic 44: Count 4
Topic 9: Count 3
Topic 29: Count 3
Topic 2: Count 2
Topic 43: Count 2
Topic 31: Count 1
Topic 21: Count 1
Topic 38: Count 1
Topic 6: Count 1
Topic 10: Count 1
Topic 5: Count 1
Most used words in Topic 16: waste, plastic, household, recycling, food, economy, product, collection, system, material
Most used words in Topic 44: waste, recycling, household, plastic, collection, product, ewaste, study, system, business
Most used words in Topic 9: waste, plastic, recycling, material, business, household, food, study, packaging, korea
Most used words in Topic 29: waste, household, food, recycling, consumer, product, plastic, collection, study, material
Most used words in Topic 2: waste, food, product, recycling, consumer, ewaste, household, study, collection, system
Most used words in Topic 43: waste, recycling, system, product, ewaste, household, study, consumer, food, management
Most used words in Topi

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Function to compute coherence score
def compute_coherence(dictionary, corpus, texts, limit, start=2, step=3):
    # YOUR CODE HERE
    coherence_values = []
    for num_topics in range(start, limit, step):
        lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = num_topics, random_state = 100)

        coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherence_model_lda.get_coherence())
    
    return coherence_values

coherence_values = compute_coherence(dictionary=corpus, corpus= doc_term_matrix, texts=processed_text, limit=50, start = 5, step=5)

optimal_nb_topics = (coherence_values.index(max(coherence_values)) + 1) * 5
print(f"the optimal number of topic for this model is: {optimal_nb_topics}")

the optimal number of topic for this model is: 45
